In [3]:
import pandas as pd
import numpy as np

In [11]:
#used on a groupedby Dealer ID group, to decide whether this contractor applied in the time window or not:
def check_app_in_window(window,DealerIDgroup):
    result=(((DealerIDgroup["quoteCreationDate"]-DealerIDgroup["Account Creation Date"])/timedelta(days=1)<window+1).any())*1
    return result

def applied_in_window_or_not(window,df_merge,df_contractors,last_updated_day):
    
    #merge - applications left join contractor:
    #df_merge=pd.merge(df_applications,df_contractors,how="left",left_on="quoteReportCode4",right_on="Dealer ID")
    #add column "Latency":
    df_merge["Latency"]=round((df_merge["quoteCreationDate"]-df_merge["Account Creation Date"])/timedelta(days=1),1)
    #drop N/A:
    df_IDandDateNoNA=df_merge.loc[df_merge["Dealer ID"].notna(),["Dealer ID","Account Creation Date","quoteCreationDate","Latency"]]
    #Use check_app_in_window to create a column for contractors, which means whether this contractor applied in the time window or not:
    df_yes_or_no=df_IDandDateNoNA.groupby(by=["Dealer ID"]).apply(lambda x: check_app_in_window(window,x))
    df_yes_or_no_reset_index=pd.DataFrame(df_yes_or_no).reset_index()
    df_yes_or_no_reset_index.rename(columns={0:"applied_in_window_or_not"},inplace=True)
    #add this column to "contractor" excel:
    df_contractors_with_applied_in_window_or_not=pd.merge(df_contractors,df_yes_or_no_reset_index,how="left",left_on="Dealer ID",right_on="Dealer ID")
    #fill N/A by 0 (because these N/As means they registered in 2016 but never applied in 2017-2019):
    df_contractors_with_applied_in_window_or_not.fillna({"applied_in_window_or_not":0},inplace=True)
    #For those whose time window due date has not arrived yet, put "Not enough info" in the "applied_in_window_or_not" column:
    df_contractors_with_applied_in_window_or_not.loc[(last_updated_day-df_contractors_with_applied_in_window_or_not["Account Creation Date"])\
                                       /timedelta(days=1)<window+1, "applied_in_window_or_not"]="Not enough info"
    
    return df_contractors_with_applied_in_window_or_not

In [12]:
df_applications=pd.read_excel("./Applications Data_01.01.16 - 09.19.19.xlsx",header=1)
df_contractors=pd.read_excel("./Contractor Onboarding Data_01.01.16 - 09.04.19.xlsx",header=14)
df_merge=pd.merge(df_applications,df_contractors,how="left",left_on="quoteReportCode4",right_on="Dealer ID")

from datetime import timedelta

window=365/2
last_updated_day=pd.to_datetime("2019-09-19")

output=applied_in_window_or_not(window,df_merge,df_contractors,last_updated_day)
output.head()

,SF ID,Dealer ID,Account Creation Date,Account Name,Mailing State,No. of Employees,Residential HVAC Sales Per Year,Years in Business,HVAC Brand #1,HVAC Brand #2,CEL MKT,Currently offers Consumer Financing?,# of Apps 2017,# of Apps 2018,# of Apps 2019,applied_in_window_or_not
0,001U000001blEDR,C15306,2016-01-04,Deliver Heating and Cooling LLC.,OH,1,"0-99,999",-,Goodman,Coleman,-,No,0,0,0,0
1,001U000001blEJq,C15309,2016-01-04,Owens' Air Conditioning & Heating Inc.,FL,4,"100,000-499,999",-,Frigidaire,Trane,-,Yes,0,0,0,0
2,001U000001blBbr,C15310,2016-01-04,Sarmiento's Heating and Cooling,IL,1,"0-99,999",-,Rheem,Aricola,-,No,0,0,0,0
3,001U000001blESp,C15311,2016-01-04,CountrySide Mechanical,SC,4,"100,000-499,999",-,Goodman,Tempstar,-,No,1,0,0,1
4,001U000001blEnM,C15312,2016-01-04,Quality Home Heating Inc,MA,3,"100,000-499,999",14,Ruud,Peerless,-,No,0,0,2,0


In [ ]:
output.to_csv("contractor_with_window_6months.csv")